In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import geohash
import random
import math
import time
import pytz
from datetime import datetime

In [2]:
# # references
# # https://github.com/nlintz/TensorFlow-Tutorials
# # https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/tutorials/mnist/mnist_softmax.py
# # https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/tutorials/mnist/fully_connected_feed.py
# # https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/tutorials/mnist/mnist.py    
# https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/tutorials/estimators/abalone.py
# https://github.com/lyft/challenge/blob/master/data_scientist/travel_time/description/travel_time.pdf
# linear regression https://aqibsaeed.github.io/2016-07-07-TensorflowLR/
# https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/2_BasicModels/linear_regression.py

In [2]:
# raw_data = {'first_name': ['Jason', 'Molly', 'Tina', 'Jake', 'Amy'],
#         'last_name': ['Miller', 'Jacobson', ".", 'Milner', 'Cooze'],
#         'age': [42, 52, 36, 24, 73],
#         'preTestScore': [4, 24, 31, ".", "."],
#         'postTestScore': ["25,000", "94,000", 57, 62, 70]}
# df = pd.DataFrame(raw_data, columns = ['first_name', 'last_name', 'age', 'preTestScore', 'postTestScore'])
# df
inputdf = pd.read_csv("train.csv")
df = inputdf

In [ ]:
full_training_data = df

In [7]:
### CONSTRUCT FINAL DATAFRAME FROM INPUT
### CONSTRUCT FINAL DATAFRAME FROM INPUT
### CONSTRUCT FINAL DATAFRAME FROM INPUT
### CONSTRUCT FINAL DATAFRAME FROM INPUT
### CONSTRUCT FINAL DATAFRAME FROM INPUT

def transform_dataset(df):
    tz = pytz.timezone('America/New_York')
    def get_local_time(timestamp):
        d = datetime.utcfromtimestamp(timestamp).replace(tzinfo=pytz.utc)
        dt = d.astimezone(tz)
        return dt
    
#     d["geo"] = d["start_geohash"] + ":" + d["end_geohash"]
    start_time = df["start_timestamp"].apply(get_local_time)
    start_time.name = 'start_timestamp_obj'
    df = pd.concat([df, start_time], axis = 1)    
    # start_time[:10]

    hour = df["start_timestamp_obj"].dt.hour
    hour.name = 'hour'
    df = pd.concat([df, hour], axis = 1)

    minute = df["start_timestamp_obj"].dt.minute
    minute.name = 'minute'
    df = pd.concat([df, minute], axis = 1)

    weekday = df["start_timestamp_obj"].dt.weekday
    weekday.name = 'weekday'
    df = pd.concat([df, weekday], axis = 1)

    del start_time
    del hour
    del minute
    del weekday
    outdf = df.copy()
    outdf.drop('start_timestamp', axis=1, inplace=True)
    outdf.drop('start_timestamp_obj', axis=1, inplace=True)
    return outdf
#     df = pd.concat([df, pd.DataFrame(start_time, index=df.index)], axis = 1)
dfff = pd.read_csv("test.csv")
d = dfff[:100].copy()
d = transform_dataset(d)
d[:10]

,row_id,start_lng,start_lat,end_lng,end_lat,hour,minute,weekday
0,0,-73.993111,40.724289,-74.000977,40.735222,3,13,6
1,1,-73.971924,40.762749,-73.965698,40.771427,13,2,1
2,2,-73.953247,40.765816,-73.952843,40.772453,13,2,1
3,3,-73.986618,40.739353,-73.949158,40.805161,4,8,3
4,4,-73.968864,40.757317,-73.982521,40.771305,23,9,2
5,5,-73.986465,40.739922,-73.979027,40.721848,20,41,3
6,6,-73.963791,40.763790,-73.995850,40.741783,22,37,5
7,7,-73.999825,40.743290,-74.001366,40.756424,11,58,5
8,8,-73.955147,40.765026,-73.990143,40.751049,23,22,2
9,9,-73.980156,40.760777,-73.970421,40.764828,13,56,3


In [8]:
df = transform_dataset(df)

In [10]:
# df.to_csv("train_mod.csv", index=False)
df[:1]

,row_id,start_lng,start_lat,end_lng,end_lat,duration,hour,minute,weekday
0,0,-74.009087,40.713818,-74.004326,40.719986,112,23,33,5


In [ ]:
# from collections import defaultdict
# g = set()
# dictionary = {}
# reverse_dictionary = {}
# for index, row in df.iterrows():
#     g.add(row['start_geohash'])
#     g.add(row['end_geohash'])
# for i, item in enumerate(g):
#     dictionary[i] = item
#     reverse_dictionary[item] = i

# tup = defaultdict(list)
# for index, row in df.iterrows():
#     tup[row['start_geohash']].append((row['duration'], row['end_geohash']))
# glist = list(g)
    
# #     gmap[row['start_geohash']]

In [10]:
# from collections import defaultdict
# dictionary = {}
# reverse_dictionary = {}
# start_list = list(df['start_geohash'])
# end_list = list(df['end_geohash'])
# geoset = set(start_list + end_list)

# for i, item in enumerate(geoset):
#     dictionary[i] = item
#     reverse_dictionary[item] = i


In [13]:
# tup = defaultdict(list)
# def geo_mapping(combo):
#     start, end, duration = combo.split(":")
#     tup[start].append((int(duration), end))
    
# # df["geo"] = df["start_geohash"] + ":" + df["end_geohash"] + ":" + df["duration"].astype(str)
# _ = df["geo"].apply(geo_mapping)


In [122]:
# sorted_tup = {}
# for k, v in tup.iteritems():
#     sorted_tup[k] = sorted(tup[k])[:100]


In [11]:
def dict_head(d, num=10):
    i = 0
    for k, v in d.iteritems():
        print k, v
        i += 1
        if i > num:
            break


In [123]:
# data_index = 0

In [124]:
# ### BEGINNING OF THE EMBEDDINGS MODEL

# def generate_batch(batch_size):
#     global data_index
#     batch = np.ndarray(shape=(batch_size), dtype=np.int32)
#     labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
#     dictionary[data_index]
#     i = 0
#     while i < batch_size:
#         item = data_index
#         element = sorted_tup.get(dictionary[data_index])
#         if not element:
#             data_index = (data_index + 1) % len(sorted_tup)
#             continue
#         options = sorted_tup[dictionary[data_index]]
#         choice = random.choice(options[:300])
#         batch[i] = item
#         labels[i, 0] = reverse_dictionary[choice[1]]
#         data_index = (data_index + 1) % len(sorted_tup)
#         i += 1
#     return batch, labels
# batch, labels = generate_batch(10)
# print 'batch', batch
# print 'labels', labels

batch [ 0  2  4  5  6  9 15 21 23 34]
labels [[    0]
 [ 7645]
 [    4]
 [15182]
 [20409]
 [39681]
 [   15]
 [ 6454]
 [14170]
 [ 8020]]


In [125]:
# batch_size = 512
# embedding_size = 8  # Dimension of the embedding vector.
# num_sampled = 256
# vocabulary_size = len(geoset)

# graph = tf.Graph()

# with graph.as_default():

#   # Input data.
#     train_inputs = tf.placeholder(tf.int64, shape=[batch_size])
#     train_labels = tf.placeholder(tf.int64, shape=[batch_size, 1])

#     # Ops and variables pinned to the CPU because of missing GPU implementation
#     with tf.device('/cpu:0'):
#     # Look up embeddings for inputs.
#         embeddings = tf.Variable(
#             tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
#         embed = tf.nn.embedding_lookup(embeddings, train_inputs)

#         # Construct the variables for the NCE loss
#         nce_weights = tf.Variable(
#             tf.truncated_normal([vocabulary_size, embedding_size],
#                                 stddev=1.0 / math.sqrt(embedding_size)))
#         nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

#   # Compute the average NCE loss for the batch.
#   # tf.nce_loss automatically draws a new sample of the negative labels each
#   # time we evaluate the loss.
#     loss = tf.reduce_mean(
#       tf.nn.nce_loss(weights=nce_weights,
#                      biases=nce_biases,
#                      labels=train_labels,
#                      inputs=embed,
#                      num_sampled=num_sampled,
#                      num_classes=vocabulary_size))

#     # Construct the SGD optimizer using a learning rate of 1.0.
#     optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

#     # Compute the cosine similarity between minibatch examples and all embeddings.
#     norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
#     normalized_embeddings = embeddings / norm
# #     valid_embeddings = tf.nn.embedding_lookup(
# #       normalized_embeddings, valid_dataset)
# #     similarity = tf.matmul(
# #       valid_embeddings, normalized_embeddings, transpose_b=True)
#     init = tf.global_variables_initializer()


In [ ]:
# # Step 5: Begin training.
# num_steps = 200000
# with tf.Session(graph=graph) as session:
#     # We must initialize all variables before we use them.
#     init.run()
#     print('Initialized')

#     average_loss = 0
#     for step in xrange(num_steps):
#         batch_inputs, batch_labels = generate_batch(
#             batch_size)
#         feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

#         # We perform one update step by evaluating the optimizer op (including it
#         # in the list of returned values for session.run()
#         _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
#         average_loss += loss_val

#         if step % 2000 == 0:
#             if step > 0:
#                 average_loss /= 2000
#             # The average loss is an estimate of the loss over the last 2000 batches.
#             print('Average loss at step ', step, ': ', average_loss)
#             average_loss = 0

#     final_embeddings = normalized_embeddings.eval()


Initialized
('Average loss at step ', 0, ': ', 927.4444580078125)
('Average loss at step ', 2000, ': ', 244.21554810619355)
('Average loss at step ', 4000, ': ', 21.559425556182862)
('Average loss at step ', 6000, ': ', 6.8963463888168333)
('Average loss at step ', 8000, ': ', 4.5532505497932432)
('Average loss at step ', 10000, ': ', 3.9644158512353895)
('Average loss at step ', 12000, ': ', 3.6713721165657045)
('Average loss at step ', 14000, ': ', 3.4193316249847414)
('Average loss at step ', 16000, ': ', 3.210755734682083)
('Average loss at step ', 18000, ': ', 3.0320033909082413)
('Average loss at step ', 20000, ': ', 2.8759165284633634)
('Average loss at step ', 22000, ': ', 2.7272590008974076)
('Average loss at step ', 24000, ': ', 2.6201186646223067)
('Average loss at step ', 26000, ': ', 2.5232962066531179)
('Average loss at step ', 28000, ': ', 2.4241663871407511)
('Average loss at step ', 30000, ': ', 2.3475983265042304)
('Average loss at step ', 32000, ': ', 2.2862670703530

In [21]:
# type(final_embeddings)

numpy.ndarray

In [254]:
input_batch = 32
data_size = len(df)
training_size = data_size - data_size//10
testing_size = data_size - training_size
INPUT_FEATURES = 7
# start_timestamp	duration	start_geohash	end_geohash	start_timestamp_obj	start_of_day	weekday	hour	minute
""" Generate the input feature dataset """
def get_batch(batch_size, df):
    global batch_index
    skip = random.randrange(0, batch_size)
    batch = np.ndarray(shape=(batch_size, INPUT_FEATURES), dtype=np.float32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.float32)
    forward_increment = 0
    for i in range(batch_size):
        increase_forward = random.randint(0, 3)
        forward_increment += increase_forward
        index = (batch_index + i + skip + forward_increment) % training_size
        batch[i, 0] = df.iloc[index]["weekday"]
        batch[i, 1] = df.iloc[index]["hour"]
        batch[i, 2] = df.iloc[index]["minute"]
        batch[i, 3] = df.iloc[index]["start_lat"]
        batch[i, 4] = df.iloc[index]["start_lng"]
        batch[i, 5] = df.iloc[index]["end_lat"]
        batch[i, 6] = df.iloc[index]["end_lng"]
        labels[i, 0] = df.iloc[index]["duration"]
    batch_index += batch_size % training_size
    return batch, labels

def get_all_batches(batch_size, input_df):
    num_batches = len(input_df) // batch_size
    batches = []
    index = 0
    for i in range(num_batches):
        if index + batch_size >= len(input_df):
            break
        df = input_df[index: index + batch_size]
        batch = np.ndarray(shape=(batch_size, INPUT_FEATURES), dtype=np.float32)
        labels = np.ndarray(shape=(batch_size, 1), dtype=np.float32)
        batch[:, 0] = df["weekday"]
        batch[:, 1] = df["hour"]
        batch[:, 2] = df["minute"]
        batch[:, 3] = df["start_lat"]
        batch[:, 4] = df["start_lng"]
        batch[:, 5] = df["end_lat"]
        batch[:, 6] = df["end_lng"]   
        labels[:, 0] = df["duration"]
        batches.append((batch, labels))
        index += batch_size
    return batches


def get_test_batch(batch_size, df):
    indices = []
    for i in range(batch_size):
        indices.append(random.randrange(0, len(df)))
    batch = np.ndarray(shape=(batch_size, INPUT_FEATURES), dtype=np.float32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.float32)
    for i, position in enumerate(indices):
        batch[i, 0] = df.iloc[position]["weekday"]
        batch[i, 1] = df.iloc[position]["hour"]
        batch[i, 2] = df.iloc[position]["minute"]
        batch[i, 3] = df.iloc[position]["start_lat"]
        batch[i, 4] = df.iloc[position]["start_lng"]
        batch[i, 5] = df.iloc[position]["end_lat"]
        batch[i, 6] = df.iloc[position]["end_lng"]
        labels[i, 0] = df.iloc[position]["duration"]
    return batch, labels

def get_full_test_data(limit=None):
    path = "test.csv"
    df = pd.read_csv("test.csv")
    if limit:
        df = df[:limit]
    df = transform_dataset(df)
    batch_size = len(df)
    batch = np.ndarray(shape=(batch_size, INPUT_FEATURES), dtype=np.float32)
    batch[:, 0] = df["weekday"]
    batch[:, 1] = df["hour"]
    batch[:, 2] = df["minute"]
    batch[:, 3] = df["start_lat"]
    batch[:, 4] = df["start_lng"]
    batch[:, 5] = df["end_lat"]
    batch[:, 6] = df["end_lng"]             
    return batch


In [14]:
def input_placeholders():
    with tf.name_scope('input_layer'):
        features = tf.placeholder(tf.float32, shape=[None, INPUT_FEATURES])
        train_labels = tf.placeholder(tf.float32, shape=[None, 1])
    return features, train_labels
def inference(features, hidden1_units, hidden2_units):
    """ Building the tensorflow model """
    with tf.name_scope('hidden1'):
        weights_1 = tf.Variable(
            tf.truncated_normal([INPUT_FEATURES, hidden1_units],
                                stddev=1.0 / math.sqrt(float(INPUT_FEATURES))),
            name='weights')
        biases = tf.Variable(tf.zeros([hidden1_units]),
                             name='biases')
        hidden1 = tf.nn.relu(tf.matmul(features, weights_1) + biases)
    # Hidden 2
    with tf.name_scope('hidden2'):
        weights_2 = tf.Variable(
            tf.truncated_normal([hidden1_units, hidden2_units],
                                stddev=1.0 / math.sqrt(float(hidden1_units))),
            name='weights')
        biases = tf.Variable(tf.zeros([hidden2_units]),
                             name='biases')
        hidden2 = tf.nn.relu(tf.matmul(hidden1, weights_2) + biases)
    # Linear
    with tf.name_scope('softmax_linear'):
        weights = tf.Variable(
            tf.truncated_normal([hidden2_units, 1],
                                stddev=1.0 / math.sqrt(float(hidden2_units))),
            name='weights')
        biases = tf.Variable(tf.zeros([1]),
                             name='biases')
        output_linear = tf.matmul(hidden2, weights) + biases
    return output_linear, (weights, weights_1, weights_2)

In [37]:
# def get_loss(output_linear, labels, n_samples):
#     """Calculates the loss from the output_linear and the labels.
#     Args:
#       logits: output_linear tensor, float - [batch_size, 1].
#       labels: Labels tensor, int32 - [batch_size].
#     Returns:
#       loss: Loss tensor of type float.
#     """
#     labels = tf.to_float(labels)
# #     cost = tf.reduce_mean(tf.square((output_linear - labels)/labels))
#     cost = tf.reduce_sum(tf.pow(output_linear-labels, 2))/(2*n_samples)
#     return cost

def get_loss(output_linear, labels, n_samples):
    """Calculates the loss from the output_linear and the labels.
    Args:
      logits: output_linear tensor, float - [batch_size, 1].
      labels: Labels tensor, int32 - [batch_size].
    Returns:
      loss: Loss tensor of type float.
    """
#     cost = tf.reduce_mean(tf.pow(output_linear-labels, 2))/(2*n_samples)
    cost = tf.reduce_sum(tf.abs((output_linear-labels)/labels)/(2)
#     reduction = tf.square(tf.subtract(output_linear, labels))
#     red = tf.reduce_mean(reduction)
    return cost

SyntaxError: invalid syntax (<ipython-input-37-cb188f605727>, line 26)

In [16]:

def training(loss, starter_learning_rate, decay_steps):
    """Sets up the training Ops.
    Creates a summarizer to track the loss over time in TensorBoard.
    Creates an optimizer and applies the gradients to all trainable variables.
    The Op returned by this function is what must be passed to the
    `sess.run()` call to cause the model to train.
    Args:
      loss: Loss tensor, from loss().
      learning_rate: The learning rate to use for gradient descent.
    Returns:
      train_op: The Op for training.
    """
    global_step = tf.Variable(0, name='global_step', trainable=False)
    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                               decay_steps, 0.96, staircase=True)   
    # Add a scalar summary for the snapshot loss.
    tf.summary.scalar('loss', loss)
    # Create the gradient descent optimizer with the given learning rate.
#     optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    optimizer = tf.train.AdamOptimizer(learning_rate)
    # Create a variable to track the global step.
    # Use the optimizer to apply the gradients that minimize the loss
    # (and also increment the global step counter) as a single training step.
    train_op = optimizer.minimize(loss, global_step=global_step)
    return train_op


In [17]:
def get_evaluation(output_linear, labels):

    diff = tf.abs(tf.subtract(output_linear, labels))
    return tf.reduce_mean(diff)

In [119]:
# full_test_data = batch_data
full_test_data = get_full_test_data()

In [120]:
type(full_test_data)

numpy.ndarray

In [163]:
# data_size = vocabulary_size
# training_size = vocabulary_size - vocabulary_size//10
# testing_size = data_size - training_size
final_results = None
batch_index = 0
num_steps = 50000
input_batch = 4
starter_learning_rate = 0.01
decay_steps = 1000
beta = 0.001
def start_training(num_steps):
#     sess = tf.InteractiveSession()
#     tf.global_variables_initializer().run()
    with tf.Graph().as_default():
        features, labels = input_placeholders()
        linear_inference, (w, w1, w2) = inference(features, 16, 8)
        loss = get_loss(linear_inference, labels, training_size)
        reg_loss = loss + beta*tf.nn.l2_loss(w1) + beta*tf.nn.l2_loss(w2)
        train_op = training(reg_loss, starter_learning_rate, decay_steps)
        evaluation = get_evaluation(linear_inference, labels)
        summary = tf.summary.merge_all()
        init = tf.global_variables_initializer()
        saver = tf.train.Saver()
        sess = tf.Session()
        summary_writer = tf.summary.FileWriter("/tmp/rides/", sess.graph)
        sess.run(init)
        training_data = df[:training_size]
        test_data = df[testing_size + 1:]
        avg_loss = 0
        validation_data, validation_labels = get_test_batch(10000, test_data)
        for step in xrange(num_steps):
            start_time = time.time()
            batch_data, label_data = get_batch(input_batch, training_data)
            feed_dict = {features:batch_data, labels:label_data}
            _, loss_value = sess.run([train_op, reg_loss],
                               feed_dict=feed_dict)
            duration = time.time() - start_time
            avg_loss += loss_value

          # Write the summaries and print an overview fairly often.
            if step % 2000 == 0:
                # Print status to stdout.
                print('Step %d: avg loss = %.5f (%.3f sec)' % (step, avg_loss/2000, duration))
                avg_loss = 0
                # Update the events file.
#                 summary_str = sess.run(summary, feed_dict=feed_dict)
#                 summary_writer.add_summary(summary_str, step)
#                 summary_writer.flush()
            if step % 6000 == 0:
#                 test_batch_data, test_label_data = get_test_batch(5000, training_data)
                feed_dict = {features:validation_data, labels:validation_labels}
                results = sess.run(evaluation, feed_dict=feed_dict)
                print 'Mean difference in testing data : %s' % results
#         batch_data = get_full_test_data()
        feed_dict = {features:full_test_data}
        global final_results
        final_results = sess.run(linear_inference, feed_dict=feed_dict)
start_training(num_steps)


Step 0: avg loss = 0.00004 (0.017 sec)
Mean difference in testing data : 846.172


KeyboardInterrupt: 

In [ ]:
# Batch normalization reference
# http://ruishu.io/2016/12/27/batchnorm/
# http://r2rt.com/implementing-batch-normalization-in-tensorflow.html#making-predictions-with-the-model

In [197]:
def dense(x, size, scope):
    return tf.contrib.layers.fully_connected(x, size, 
                                             activation_fn=None,
                                             scope=scope)

def dense_batch_relu(input_num, output_num, input_mat, phase, scope):
    with tf.name_scope(scope):
        weights_1 = tf.Variable(
            tf.truncated_normal([input_num, output_num],
                                stddev=1.0 / math.sqrt(float(input_num))),
            name='weights')
        biases = tf.Variable(tf.zeros([output_num]),
                             name='biases')
        hidden1 = tf.nn.relu(tf.matmul(input_mat, weights_1) + biases)
        return hidden1
        
# def dense_batch_relu(x, phase, scope):
#     with tf.variable_scope(scope):
# #         h1 = tf.contrib.layers.fully_connected(x, 64, 
# #                                                activation_fn=None,
# #                                                scope='dense')
#         weights_1 = tf.Variable(
#             tf.truncated_normal([x, 32],
#                                 stddev=1.0 / math.sqrt(float(x))),
#             name='weights')
#         biases = tf.Variable(tf.zeros([32]),
#                              name='biases')
#         hidden1 = tf.nn.relu(tf.matmul(features, weights_1) + biases)        
# #         h2 = tf.contrib.layers.batch_norm(h1, 
# #                                           center=True, scale=True, 
# #                                           is_training=phase,
# #                                           scope='bn')
#         return hidden1

In [198]:
input_batch = 64
tf.reset_default_graph()
beta = tf.constant(0.01, dtype='float32')
train_batch = tf.placeholder('float32', (None, INPUT_FEATURES), name='train_batch')
y = tf.placeholder('float32', (None, 1), name='y')
validation_labels = tf.placeholder('float32', (None, 1), name='validation_labels')
test_batch = tf.placeholder('float32', (None, 1), name='test_labels')
phase = tf.placeholder(tf.bool, name='phase')
n_samples = tf.constant(64000.0, dtype='float32')
h1 = dense_batch_relu(INPUT_FEATURES, 32, train_batch, phase, 'layer1')
h2 = dense_batch_relu(32, 32, h1, phase, 'layer2')
# h1 = dense_batch_relu(train_batch, phase,'layer1')
# h2 = dense_batch_relu(h1, phase, 'layer2')
# linear = dense(h2, 1, 'linear')
linear = dense_batch_relu(32, 10, h2, phase, 'linear')

evaluation = tf.reduce_mean(tf.abs(tf.subtract(linear, validation_labels)))

with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.abs(linear-y)) + beta*tf.nn.l2_loss(linear) + beta*tf.nn.l2_loss(h1) + beta*tf.nn.l2_loss(h2)


In [106]:
training_data = df[:training_size]
test_data = df[testing_size + 1:]
batches = get_all_batches(input_batch, training_data)
batch_index = 0
batch_list_len = len(batches)

In [200]:
validation_data, validation_labels = get_test_batch(10000, test_data)

def train():
#     update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
#     with tf.control_dependencies(update_ops):
        # Ensures that we execute the update_ops before performing the train_step
#         train_step = tf.train.AdamOptimizer(1.0).minimize(loss)
    train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)        
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
#     training_data = df[:training_size]
#     batches = get_all_batches(input_batch, training_data)
#     batch_index = 0
#     batch_list_len = len(batches)
#     test_data = df[testing_size + 1:]
    avg_loss = 0
#     validation_data, validation_labels = get_test_batch(10000, test_data)
#     iterep = len(df)//(input_batch*500)
    iterep = 20000
    print 'iterep ', iterep
    for i in range(iterep * 200):
#         batch_data, label_data = get_batch(input_batch, training_data)
        batch_index = i % batch_list_len
        batch_data, label_data = batches[batch_index]
        _, loss_value = sess.run([train_step, loss],
                 feed_dict={'train_batch:0': batch_data, 
                            'y:0': label_data, 
                            'phase:0': 1})
        avg_loss += loss_value
        if (i + 1) %  4000 == 0:
            print('Step %d: avg loss = %.4f' % (i, avg_loss/4000))            
            avg_loss = 0
        if (i + 1) %  10000 == 0: 
            t = sess.run(evaluation, 
                         feed_dict={'train_batch:0': validation_data,
                                    'validation_labels:0': validation_labels,
                                    'phase:0': 0})
            print('Step %d: evaluation on validation set = %0.1f' % (i, t))
    final_results = sess.run(linear, 
                 feed_dict={'train_batch:0': full_test_data,
                            'phase:0': 0})
    return final_results
final_results = train()

iterep  20000
Step 3999: avg loss = 70874.4176
Step 7999: avg loss = 743.3454


UnboundLocalError: local variable 'h1' referenced before assignment

In [86]:
final_results[:2]

array([[ 822.66094971],
       [ 822.66094971]], dtype=float32)

In [94]:
index = [i for i in range(0, len(final_results))]
final_output = pd.DataFrame({'row_id': index})
final_output['duration'] = final_results[:,0]
final_output['duration'] = final_output['duration'].astype(int)
final_results[:1]

array([[ 740.24932861]], dtype=float32)

In [95]:

final_output.to_csv("duration.csv", index=False)

In [42]:
final_output[100:110]

,row_id,duration
100,101,676
101,102,666
102,103,593
103,104,675
104,105,529
105,106,610
106,107,681
107,108,595
108,109,346
109,110,581


In [ ]:
# https://gist.github.com/mchirico/bcc376fb336b73f24b29

In [261]:
def assert_geohashes_in_set(df, reverse_dictionary):
    for index, row in df.iterrows():
        missing = set()
        if not reverse_dictionary.get(row["start_geohash"]):
            missing.add(row["start_geohash"])
        if not reverse_dictionary.get(row["end_geohash"]):
            missing.add(row["end_geohash"])
        if len(missing) > 0:
            raise Exception("end_geohash is not in global dataset: %s"% len(missing))

def transform_input():
    df = pd.read_csv("test.csv")
    df['start_geohash'] = df.apply(geohash_encode_start, axis=1)
    print 'end geohash'
    df['end_geohash'] = df.apply(geohash_encode_end, axis=1)    
#     df["start_timestamp_obj"] = df.apply(get_local_time, axis=1)
#     df.to_csv("test_mod.csv")
#     df["start_of_day"] = df.apply(get_start_of_the_day, axis=1)
#     df["weekday"] = df.apply(get_weekday, axis=1)
#     df["hour"] = df.apply(get_hour, axis=1)
#     df["minute"] = df.apply(get_min, axis=1)
    return df

def handle_test_data():
    df = pd.read_csv("test.csv")
    df = transform_input(df)
    assert_geohashes_in_set(df, reverse_dictionary)
    return df

In [266]:
# dftest = transform_input()
# assert_geohashes_in_set(dftest, reverse_dictionary)
assert_geohashes_in_set(dftest, reverse_dictionary)

Exception: end_geohash is not in global dataset: 1

In [262]:
def input_placeholders():
    with tf.name_scope('input_layer'):
        features = tf.placeholder(tf.float32, shape=[None, INPUT_FEATURES])
        train_labels = tf.placeholder(tf.float32, shape=[None, 1])
    return features, train_labels
def inference(features, hidden1_units, hidden2_units, hidden3_units):
    """ Building the tensorflow model """
    with tf.name_scope('hidden1'):
        weights_1 = tf.Variable(
            tf.truncated_normal([INPUT_FEATURES, hidden1_units],
                                stddev=1.0 / math.sqrt(float(INPUT_FEATURES))),
            name='weights')
        biases = tf.Variable(tf.zeros([hidden1_units]),
                             name='biases')
        hidden1 = tf.nn.relu(tf.matmul(features, weights_1) + biases)
    # Hidden 2
    with tf.name_scope('hidden2'):
        weights_2 = tf.Variable(
            tf.truncated_normal([hidden1_units, hidden2_units],
                                stddev=1.0 / math.sqrt(float(hidden1_units))),
            name='weights')
        biases = tf.Variable(tf.zeros([hidden2_units]),
                             name='biases')
        hidden2 = tf.nn.relu(tf.matmul(hidden1, weights_2) + biases)
    # Hidden 3
    with tf.name_scope('hidden2'):
        weights_3 = tf.Variable(
            tf.truncated_normal([hidden2_units, hidden3_units],
                                stddev=1.0 / math.sqrt(float(hidden1_units))),
            name='weights')
        biases = tf.Variable(tf.zeros([hidden3_units]),
                             name='biases')
        hidden3 = tf.nn.relu(tf.matmul(hidden2, weights_3) + biases)
        # Linear
    with tf.name_scope('softmax_linear'):
        weights = tf.Variable(
            tf.truncated_normal([hidden3_units, 1],
                                stddev=1.0 / math.sqrt(float(hidden3_units))),
            name='weights')
        biases = tf.Variable(tf.zeros([1]),
                             name='biases')
        output_linear = tf.matmul(hidden3, weights) + biases
    return output_linear, (weights, weights_1, weights_2)

def get_loss(output_linear, labels, n_samples):
#     cost = tf.reduce_mean(tf.pow(output_linear-labels, 2))/(2*n_samples)
    cost = tf.reduce_sum(tf.square(output_linear-labels))/(400000)
#     reduction = tf.square(tf.subtract(output_linear, labels))
#     red = tf.reduce_mean(reduction)
    return cost

def training(loss, starter_learning_rate, decay_steps):
    global_step = tf.Variable(0, name='global_step', trainable=False)
    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                               decay_steps, 0.96, staircase=False)   
    tf.summary.scalar('loss', loss)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train_op = optimizer.minimize(loss, global_step=global_step)
    return train_op


def get_evaluation(output_linear, labels):

    diff = tf.abs(tf.subtract(output_linear, labels))
    return tf.reduce_mean(diff)

In [255]:
training_data = df[:training_size]
test_data = df[testing_size + 1:]
input_batch = 20000
batches = get_all_batches(input_batch, training_data[:500000])
batch_index = 0
batch_list_len = len(batches)
validation_data, validation_labels = get_test_batch(10000, training_data)

In [ ]:
# data_size = vocabulary_size
# training_size = vocabulary_size - vocabulary_size//10
# testing_size = data_size - training_size
final_results = None
batch_index = 0
num_steps = 5000000
starter_learning_rate = 0.0001
decay_steps = 100000
beta = 0.001
def start_training(num_steps):
#     sess = tf.InteractiveSession()
#     tf.global_variables_initializer().run()
    with tf.Graph().as_default():
        features, labels = input_placeholders()
        linear_inference, (w, w1, w2) = inference(features, 500, 500, 500)
        loss = get_loss(linear_inference, labels, training_size)
        reg_loss = loss + beta*tf.nn.l2_loss(w1) + beta*tf.nn.l2_loss(w2)
        train_op = training(reg_loss, starter_learning_rate, decay_steps)
        evaluation = get_evaluation(linear_inference, labels)
#         summary = tf.summary.merge_all()
        init = tf.global_variables_initializer()
#         saver = tf.train.Saver()
        sess = tf.Session()
#         summary_writer = tf.summary.FileWriter("/tmp/rides/", sess.graph)
        sess.run(init)
        avg_loss = 0
        for step in xrange(num_steps):
            start_time = time.time()
            batch_i = step % batch_list_len
#             batch_data, label_data = batches[batch_i]
            batch_data, label_data = get_batch(10000, training_data)
            feed_dict = {features:batch_data, labels:label_data}
            _, loss_value = sess.run([train_op, reg_loss],
                               feed_dict=feed_dict)
            duration = time.time() - start_time
            avg_loss += loss_value

          # Write the summaries and print an overview fairly often.
            if step % 2000 == 0:
                # Print status to stdout.
                print('Step %d: avg loss = %.5f (%.3f sec)' % (step, avg_loss/2000, duration))
                avg_loss = 0
                # Update the events file.
#                 summary_str = sess.run(summary, feed_dict=feed_dict)
#                 summary_writer.add_summary(summary_str, step)
#                 summary_writer.flush()
            if step % 10000 == 0:
#                 test_batch_data, test_label_data = get_test_batch(5000, training_data)
                feed_dict = {features:validation_data, labels:validation_labels}
                results = sess.run(evaluation, feed_dict=feed_dict)
                print 'Mean difference in testing data : %s' % results
#         batch_data = get_full_test_data()
        feed_dict = {features:full_test_data}
        global final_results
        final_results = sess.run(linear_inference, feed_dict=feed_dict)
start_training(num_steps)